In [1]:
import wget
import os.path
import json


import numpy as np
from numpy.linalg import det, inv
from numpy import array
import random

import ase.db
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
from anneling_super_cell import super_cell

In [3]:
DATA_PATH = "database"
database_name1 = "c2db-20211702.db"
database_name2 = "c2db-20211702.db"

In [4]:
def get_data_as_pd(path,options,props):
    db = ase.db.connect(path)
    rows = db.select(options)
    data = [ [ x.get(p) for p in props ] for x in rows]
    raw_df = pd.DataFrame( data, columns = props);
    return raw_df 

In [5]:
path = DATA_PATH+database_name1

options = 'is_magnetic=True, thermodynamic_stability_level=3'
props = ["formula","spgnum", "spacegroup","uid","asr_id"] 

raw_df_1 = get_data_as_pd(path,options,props)
raw_df_1 = raw_df_1[ (raw_df_1["spacegroup"]!='P1') & (raw_df_1["spacegroup"]!='Pc')& (raw_df_1["spacegroup"]!='P-1') ];


#Print
print("The number of elements is:",len(list(raw_df_1)) )
print( raw_df_1[["formula","uid"]] )

The number of elements is: 5
Empty DataFrame
Columns: [formula, uid]
Index: []


In [6]:
path = DATA_PATH+database_name2

options = 'thermodynamic_stability_level=3'
props = ["formula","spgnum", "spacegroup","gap","uid"] 

raw_df_2 = get_data_as_pd(path,options,props)
raw_df_2 = raw_df_2[ (raw_df_2["gap"]<=10e-3)& (raw_df_2["spacegroup"]!='P1') & (raw_df_2["spacegroup"]!='Pc')& (raw_df_2["spacegroup"]!='P-1') ];


#Print
print("The number of elements is:",len(list(raw_df_2)) )
print( raw_df_2[["formula","uid","gap"]] )

The number of elements is: 5
Empty DataFrame
Columns: [formula, uid, gap]
Index: []


In [7]:
# get jsons 
def extract_json(uid_list,save_path ="JSONcolection"):
    for uid in uid_list:
        dataurl = 'https://cmrdb.fysik.dtu.dk/c2db/row/'+uid+'/all_data';
        file = save_path+"/"+uid+"_data.json"
        if os.path.isfile(file):
            print("file: ",file, "found")
        else:
            print(wget.download(dataurl, out="./"+save_path))

def extract_structure(uid_list ,save_path="XYZcolection"):
    for uid in uid_list:
        
        #https://cmrdb.fysik.dtu.dk/c2db/row/N2O2V3-358facb64a22/data/structure.json
        dataurl = 'https://cmrdb.fysik.dtu.dk/c2db/row/'+uid+'/data/structure.json';
        print(dataurl)
        file = save_path+"/"+uid+".json"
        dw_path = save_path+"/"+"structure.json"
        if os.path.isfile(file):
            print("file: ",file, "found")
        else:
            print(wget.download(dataurl, out="./"+save_path))
            os.rename(dw_path, file)

In [8]:
print("extract_data1:")
print(raw_df_1["uid"][:1])
extract_structure(raw_df_1["uid"][:10])
print("extract_data2:")
extract_structure(raw_df_1["uid"][:10])

extract_data1:
Series([], Name: uid, dtype: object)
extract_data2:


In [9]:
# Example of good practice in data hadeling 
# uid= "C2-a6735a4a3797"
# with open('XYZcolection/structure.json', 'r') as file:
#     json_data= json.load(file)
# mytype= "__ndarray__";
# datastruc=json_data['1']["cell"]["array"];
# if mytype in datastruc:
#     shape= datastruc[mytype][0]
#     dtype= datastruc[mytype][1]
#     lattice = array(datastruc[mytype][2],dtype=dtype).reshape(shape)


In [10]:
#f = "/Users/voicutu/Documents/GitHub/vdw_evolve/sandbox/XYZcolection/c2db-822.xyz"

def get_lattice_from_structure(f):
    with open(f, 'r') as file:
        json_data= json.load(file)
    mytype= "__ndarray__";
    datastruc=json_data['1']["cell"]["array"];
    if mytype in datastruc:
        shape= datastruc[mytype][0]
        dtype= datastruc[mytype][1]
        lattice = array(datastruc[mytype][2],dtype=dtype).reshape(shape)
    return lattice 

In [11]:
#
up = 99999999
bond =20
model_par = {
    
    'initialTemp': 4, 
    'finalTemp': 0.0002,

    'beta': 10,
    'bounds': [[-2*bond,2*bond] for _ in range(4)],

    'nr_neighbours': 1,
    'step_size': 4,
    'gaussian_sd':3,
    
    'known_min': -up,
    "start_point":[1,5,3,2],
    
    "strain_boundary":[[-0.2,0.2],[-0.2,0.2]]
}



In [12]:
uids=["C2-a6735a4a3797","WS2-64090c9845f8"]
extract_structure(uids)

https://cmrdb.fysik.dtu.dk/c2db/row/C2-a6735a4a3797/data/structure.json
file:  XYZcolection/C2-a6735a4a3797.json found
https://cmrdb.fysik.dtu.dk/c2db/row/WS2-64090c9845f8/data/structure.json
file:  XYZcolection/WS2-64090c9845f8.json found


In [13]:
# xyz path 

structure_path = "XYZcolection"#/Users/voicutu/Documents/GitHub/vdw_evolve/sandbox/XYZcolection"
uid_1 = "WS2-64090c9845f8"#"VCl2-1ccb0b28392d"
uid_2 = "C2-a6735a4a3797"#"Mn2Se2-a1d595c25fc5"


path1=structure_path+"/"+uid_1+".json"
path2=structure_path+"/"+uid_2+".json"

# lattices
latice1 = get_lattice_from_structure(path1)
latice2 = get_lattice_from_structure(path2)


cel1 = np.array([[latice1[0][0], latice1[1][0]],
        [latice1[0][1], latice1[1][1]]])

cel2 = np.array([[latice2[0][0], latice2[1][0]],
        [latice2[0][1], latice2[1][1]]])

tA, tB, t_cel2_no_strain, diagonal_strain, strain = super_cell(cel1,cel2, 7, model_par)

new = np.dot(tA, cel1)

print("\ncel1",cel1)
print("cel2",cel2)
print("\n _____\n tA:\n",tA)
print("det:",det(tA))
print("\n _____\n tB:\n",tB)
print("\n _____\n t_cel2_no_strain:\n",t_cel2_no_strain)
print("\n _____\n Strain diagonal :\n",diagonal_strain)
print("\n _____\n Strain:\n",strain)

super_cell= np.dot(tA,cel1)
print("\n _____\n Super cell:\n",super_cell)
print("det:",det(super_cell))





# print("\n zero_mat:\n",zero_mat)

temp:4|epoch:0|change:203152658542.05814|value:81039601249.60513
temp:4|epoch:5|change:47188541507.79807|value:54431774794.87057
temp:0.0975609756097561|epoch:0|change:274327249581.25876|value:54431774794.87057
temp:0.0975609756097561|epoch:5|change:139813375841.45203|value:54431774794.87057
temp:0.04938271604938272|epoch:0|change:237323392125.22183|value:54431774794.87057
temp:0.04938271604938272|epoch:5|change:298388666163.77124|value:54431774794.87057
temp:0.03305785123966942|epoch:0|change:168296520768.3377|value:54431774794.87057
temp:0.03305785123966942|epoch:5|change:168320977983.9088|value:54431774794.87057
temp:0.02484472049689441|epoch:0|change:1162517115521.8184|value:54431774794.87057
temp:0.02484472049689441|epoch:5|change:42974503339.093025|value:10430171715.20872
temp:0.01990049751243781|epoch:0|change:80753524666.94524|value:10430171715.20872
temp:0.01990049751243781|epoch:5|change:291866735244.30786|value:10430171715.20872
temp:0.016597510373443983|epoch:0|change:88811

temp:0.0020397756246812856|epoch:5|change:18701040459.85303|value:4833067932.959404
temp:0.0019990004997501253|epoch:0|change:69845136393.75714|value:4833067932.959404
temp:0.0019990004997501253|epoch:5|change:59813452673.23213|value:4833067932.959404
temp:0.0019598236158745717|epoch:0|change:135619207746.76065|value:4833067932.959404
temp:0.0019598236158745717|epoch:5|change:2252850047960.1733|value:4833067932.959404
temp:0.0019221528111484868|epoch:0|change:13172105924191.105|value:4833067932.959404
temp:0.0019221528111484868|epoch:5|change:402670495964.5369|value:4833067932.959404
temp:0.0018859028760018863|epoch:0|change:354115108301.3782|value:4833067932.959404
temp:0.0018859028760018863|epoch:5|change:37763096615.57325|value:4833067932.959404
temp:0.0018509949097639986|epoch:0|change:2035889801520.0283|value:4833067932.959404
temp:0.0018509949097639986|epoch:5|change:3593806728617.463|value:4833067932.959404
temp:0.0018173557473875517|epoch:0|change:91095028875.19614|value:483306

temp:0.0010098459984852306|epoch:5|change:8501894104527.465|value:2092864872.1239257
temp:0.0009997500624843783|epoch:0|change:685040202028.9598|value:2092864872.1239257
temp:0.0009997500624843783|epoch:5|change:246747753683.6869|value:2092864872.1239257
temp:0.0009898539965355103|epoch:0|change:178005304819.87225|value:2092864872.1239257
temp:0.0009898539965355103|epoch:5|change:1492825192798.7896|value:2092864872.1239257
temp:0.0009801519235481493|epoch:0|change:1394378196553.5618|value:2092864872.1239257
temp:0.0009801519235481493|epoch:5|change:51366655025.92797|value:2092864872.1239257
temp:0.0009706381946129574|epoch:0|change:158946515740.35562|value:2092864872.1239257
temp:0.0009706381946129574|epoch:5|change:239008926029.4035|value:2092864872.1239257
temp:0.0009613073780341257|epoch:0|change:3272809171312.718|value:2092864872.1239257
temp:0.0009613073780341257|epoch:5|change:1904586821258.838|value:2092864872.1239257
temp:0.0009521542489883355|epoch:0|change:14069169953219.29|v

temp:0.0006755615605472042|epoch:0|change:467469157031.55273|value:2092864872.1239257
temp:0.0006755615605472042|epoch:5|change:107680018253.21512|value:2092864872.1239257
temp:0.000671028350947827|epoch:0|change:19657232219408.008|value:2092864872.1239257
temp:0.000671028350947827|epoch:5|change:895172693730.1696|value:2092864872.1239257
temp:0.0006665555740709876|epoch:0|change:148786270410.6831|value:2092864872.1239257
temp:0.0006665555740709876|epoch:5|change:154998377353.95944|value:2092864872.1239257
temp:0.0006621420294653197|epoch:0|change:408566220438.893|value:2092864872.1239257
temp:0.0006621420294653197|epoch:5|change:196697206756.79358|value:2092864872.1239257
temp:0.0006577865482650873|epoch:0|change:189784096516.79108|value:2092864872.1239257
temp:0.0006577865482650873|epoch:5|change:951631640824.6655|value:2092864872.1239257
temp:0.0006534879921581434|epoch:0|change:71193220507.18765|value:2092864872.1239257
temp:0.0006534879921581434|epoch:5|change:296864874232.74414|v

temp:0.0005075498033244512|epoch:5|change:342396505379.338|value:594166264.3131534
temp:0.0005049867440979674|epoch:0|change:225051821299.84848|value:594166264.3131534
temp:0.0005049867440979674|epoch:5|change:342256944867.9854|value:594166264.3131534
temp:0.0005024494410249969|epoch:0|change:175902823478.49838|value:594166264.3131534
temp:0.0005024494410249969|epoch:5|change:631432008210.17|value:594166264.3131534
temp:0.0004999375078115236|epoch:0|change:410399955182.33984|value:594166264.3131534
temp:0.0004999375078115236|epoch:5|change:37529461920.45174|value:594166264.3131534
temp:0.0004974505658500187|epoch:0|change:4741414829632.922|value:594166264.3131534
temp:0.0004974505658500187|epoch:5|change:435607697984.4179|value:594166264.3131534
temp:0.0004949882440292043|epoch:0|change:21225518316.70301|value:594166264.3131534
temp:0.0004949882440292043|epoch:5|change:100184618388570.17|value:594166264.3131534
temp:0.0004925501785494397|epoch:0|change:34570971684.76973|value:594166264

...

temp:0.0003389543259045849|epoch:5|change:312908403528.4049|value:594166264.3131534
temp:0.00033780930664639865|epoch:0|change:1410929666530.6135|value:594166264.3131534
temp:0.00033780930664639865|epoch:5|change:100245564769088.44|value:594166264.3131534
temp:0.0003366719973066246|epoch:0|change:264658312764.06143|value:594166264.3131534
temp:0.0003366719973066246|epoch:5|change:853961404532.8702|value:594166264.3131534
temp:0.00033554232027514525|epoch:0|change:1310048194737.6663|value:594166264.3131534
temp:0.00033554232027514525|epoch:5|change:117218117442.3675|value:594166264.3131534
temp:0.0003344201989800189|epoch:0|change:280965094754.5568|value:594166264.3131534
temp:0.0003344201989800189|epoch:5|change:284332794432.5162|value:594166264.3131534
temp:0.000333305557870178|epoch:0|change:296054931893.58984|value:594166264.3131534
temp:0.000333305557870178|epoch:5|change:887179795200.2942|value:594166264.3131534
temp:0.0003321983223984724|epoch:0|change:955043858958.8052|value:594

temp:0.0002906765496693556|epoch:0|change:59192232008.60892|value:594166264.3131534
temp:0.0002906765496693556|epoch:5|change:242000491642.54755|value:594166264.3131534
temp:0.0002898340699949281|epoch:0|change:1093250944744.552|value:594166264.3131534
temp:0.0002898340699949281|epoch:5|change:479226664127.6857|value:594166264.3131534
temp:0.00028899645979336774|epoch:0|change:808882157830.5894|value:594166264.3131534
temp:0.00028899645979336774|epoch:5|change:388094020920.9333|value:594166264.3131534
temp:0.0002881636769685183|epoch:0|change:468160449199.6624|value:594166264.3131534
temp:0.0002881636769685183|epoch:5|change:180255858190.634|value:594166264.3131534
temp:0.0002873356799080528|epoch:0|change:372564200198.35284|value:594166264.3131534
temp:0.0002873356799080528|epoch:5|change:286016482463.2278|value:594166264.3131534
temp:0.000286512427476542|epoch:0|change:2151419283488.7876|value:594166264.3131534
temp:0.000286512427476542|epoch:5|change:504122108437.30646|value:5941662

temp:0.0002537910031089397|epoch:5|change:8977998531977.79|value:594166264.3131534
temp:0.0002531485349028542|epoch:0|change:19039411890868.168|value:594166264.3131534
temp:0.0002531485349028542|epoch:5|change:1281249224773.7537|value:594166264.3131534
temp:0.0002525093112808534|epoch:0|change:230315049151.72076|value:594166264.3131534
temp:0.0002525093112808534|epoch:5|change:226150974408.4581|value:594166264.3131534
temp:0.00025187330772621366|epoch:0|change:7039871520179.028|value:594166264.3131534
temp:0.00025187330772621366|epoch:5|change:463502407749.80524|value:594166264.3131534
temp:0.00025124049996859486|epoch:0|change:338596392866.5745|value:594166264.3131534
temp:0.00025124049996859486|epoch:5|change:277899031559.64343|value:594166264.3131534
temp:0.0002506108639809535|epoch:0|change:854300551836.7941|value:594166264.3131534
temp:0.0002506108639809535|epoch:5|change:1485015303469.8062|value:594166264.3131534
temp:0.0002499843759765014|epoch:0|change:95712265892.93626|value:5

temp:0.00022521254433871966|epoch:5|change:540946714879.13275|value:594166264.3131534
temp:0.00022470647716420425|epoch:0|change:2240655417827.6997|value:594166264.3131534
temp:0.00022470647716420425|epoch:5|change:1096864645202.9178|value:594166264.3131534
temp:0.00022420267922201672|epoch:0|change:177144582536.72272|value:594166264.3131534
temp:0.00022420267922201672|epoch:5|change:320083616749.9456|value:594166264.3131534
temp:0.0002237011352832616|epoch:0|change:263823702199.5871|value:594166264.3131534
temp:0.0002237011352832616|epoch:5|change:139298273092.47995|value:594166264.3131534
temp:0.0002232018302550081|epoch:0|change:255203938604.5842|value:594166264.3131534
temp:0.0002232018302550081|epoch:5|change:517321046357.10706|value:594166264.3131534
temp:0.00022270474917877623|epoch:0|change:57741780817.075455|value:594166264.3131534
temp:0.00022270474917877623|epoch:5|change:563810404783.7325|value:594166264.3131534
temp:0.0002222098772290428|epoch:0|change:498075432614.5805|va

temp:0.00020200999949497489|epoch:0|change:739336916102.5317|value:594166264.3131534
temp:0.00020200999949497489|epoch:5|change:1507952860360.132|value:594166264.3131534
temp:0.00020160274179728833|epoch:0|change:5148599035240.992|value:594166264.3131534
temp:0.00020160274179728833|epoch:5|change:128293175518.0027|value:594166264.3131534
temp:0.0002011971228811427|epoch:0|change:2813283146384.911|value:594166264.3131534
temp:0.0002011971228811427|epoch:5|change:339564165003.2885|value:594166264.3131534
temp:0.0002007931328748556|epoch:0|change:107905334692.87381|value:594166264.3131534
temp:0.0002007931328748556|epoch:5|change:7917313607731.583|value:594166264.3131534
temp:0.00020039076198587235|epoch:0|change:939102620504.6392|value:594166264.3131534
temp:0.00020039076198587235|epoch:5|change:127287025188.17145|value:594166264.3131534

cel1 [[ 3.18594595 -1.59297297]
 [ 0.          2.75911013]]
cel2 [[ 2.46723126e+00 -1.23361563e+00]
 [-1.75506857e-20  2.13668495e+00]]

 _____
 tA:
 [

In [14]:
solutions = {"uid_1":[], "uid_2":[],
             "mat_1":[], "mat_2":[],
             "tA":[],    "tB":[],
             "new":[],   "strainB":[],
             "new_uid":[],       }

In [15]:
structure_path = "/Users/voicutu/Documents/GitHub/vdw_evolve/sandbox/XYZcolection"
saving_spot = 5
to_be_generated = 10
while to_be_generated!=0:
    
    data1= raw_df_1.iloc[random.randinit(0,len(raw_df_1["uid"]))
    data2= raw_df_1.iloc[random.randinit(0,len(raw_df_2["uid"]))
    uid_1 = data1["uid"]
    uid_2 = data2["uid"]
    
    new_uid =uid_1+"_"+uid_2
    
    if new_uid not in solutions["new_uid"]:
        to_be_generated = to_be_generated-1
        
        solutions["new_uid"].append(new_uid)
        solutions["uid_1"].append(uid_1)
        solutions["uid_2"].append(uid_2)
        
        # calculate the super cel 
        
        # xyz path 
        path1=structure_path+"/"+uid_1+".json"
        path2=structure_path+"/"+uid_2+".json"
        
        # lattices
        latice1 = get_lattice_from_structure(path1)
        latice2 = get_lattice_from_structure(path2)


        cel1 = [[latice1[0][0], latice1[1][0]],
                [latice1[0][1], latice1[1][1]]]

        cel2 = [[latice2[0][0], latice2[1][0]],
                [latice2[0][1], latice2[1][1]]]
        
        tA, tB, strain = build_annealing(cel1,cel2, 5, model_par)
        new = np.dot(tA, cel1)
        
        solutions["tA"].append(tA)
        solutions["tB"].append(tB)
        solutions["new"].append(new)
        solutions["strainB"].append(strain)
    
    # save
    if k%saving_spot==0:
        data = pd.DataFrame.from_dict(solutions)
        data.to_csv("SuperCell_"+str(len(solutions["new"])))

print("Done")

SyntaxError: invalid syntax (1642023313.py, line 7)